In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data, DataLoader
import networkx as nx
import numpy as np
import random
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
import copy
import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
wandb.init(project="graph-neural-network", config={
    "learning_rate": 1e-4,
    "epochs": 200,
    "in_channels": 1,
    "hidden_channels": 16,
    "out_channels": 8,
    "heads": 4,
    "num_layers": 4, 
    "dropout":0.3
})
config = wandb.config

wandb: Currently logged in as: 2366301182. Use `wandb login --relogin` to force relogin


In [4]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads, num_layers, dropout):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels * heads, hidden_channels, heads=heads))
        self.convs.append(GATConv(hidden_channels * heads, out_channels, heads=1))

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = x.relu()
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index)
        return x
    
    def edge_prediction(self, data):
        x = self.forward(data)
        edge_index = data.edge_index.long()
        edge_scores = (x[edge_index[0]] * x[edge_index[1]]).sum(dim=1)
        return edge_scores
    
    def edge_index_prediction(self,data):
        pred = self.edge_prediction(data)
        pred = torch.round(torch.sigmoid_(pred));
        indices = torch.nonzero(pred == 1).flatten().tolist()
        return indices

In [5]:
in_channels = config.in_channels
hidden_channels = config.hidden_channels
out_channels = config.out_channels
heads = config.heads
num_layers = config.num_layers
dropout = config.dropout
learning_rate = config.learning_rate
epochs = config.epochs

In [7]:
loaded_dataset = torch.load('train_dataset')

# 划分训练集, 验证集和测试集
train_indices, val_indices = train_test_split(list(range(len(loaded_dataset))), test_size=0.2, random_state=42)

# 创建训练集和验证集的数据子集
train_dataset = torch.utils.data.Subset(loaded_dataset, train_indices)
val_dataset = torch.utils.data.Subset(loaded_dataset, val_indices)

# 训练集数据
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [8]:
model = GAT(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels, 
            heads=heads, num_layers=num_layers, dropout=dropout).to(device)

best_model = GAT(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels, 
            heads=heads, num_layers=num_layers, dropout=dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [9]:
# 训练模型
best_val_loss = 999
early_stop_count = 0
patience = 20  # 当超出20次都没有出现新的best_val_loss时，模型自动提前停止训练。

In [10]:
for epoch in range(epochs):
    epoch_loss = 0  
    epoch_val_loss = 0
    
    model.train()
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = model.edge_prediction(batch)
        loss = loss_fn(pred, batch.y)
        epoch_loss += loss.item()/len(train_loader) 
        loss.backward()
        optimizer.step()
    
    model.eval()
    for batch_v in val_loader:
        batch_v = batch_v.to(device)
        optimizer.zero_grad()
        pred_v = model.edge_prediction(batch_v)
        loss_v = loss_fn(pred_v, batch_v.y)
        epoch_val_loss += loss_v.item()/len(val_loader) 
        
    
    wandb.log({"train_loss": epoch_loss, "val_loss": epoch_val_loss})

    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss}, Val_Loss: {epoch_val_loss}')
    
    if not best_val_loss or (epoch_val_loss < best_val_loss):
        best_val_loss = epoch_val_loss
        print(f'best_val_loss = {epoch_val_loss}')
        best_model.load_state_dict(copy.deepcopy(model.state_dict()))
        best_epoch = epoch
        early_stop_count = 0
    else:
        early_stop_count += 1
    
    if early_stop_count >= patience or epoch >= epochs - 1:
        print('Early Stopping! \nBecause %d epochs the accuracy have no improvement.'%(patience))
        print('Best testing Loss = %4.2f'%(best_val_loss))
        print('Best t = %3g'%(best_epoch))
        # test_net.load_net_params(best_model)
        break

Epoch 1/200, Loss: 0.693147519707681, Val_Loss: 0.6931475095748889
best_val_loss = 0.6931475095748889
Epoch 2/200, Loss: 0.6931475124359139, Val_Loss: 0.6931475086212147
best_val_loss = 0.6931475086212147
Epoch 3/200, Loss: 0.693147509217263, Val_Loss: 0.6931475081443774
best_val_loss = 0.6931475081443774
Epoch 4/200, Loss: 0.6931475117206581, Val_Loss: 0.6931475081443774
Epoch 5/200, Loss: 0.6931475111246117, Val_Loss: 0.6931475081443774
Epoch 6/200, Loss: 0.693147508859635, Val_Loss: 0.6931475081443774
Epoch 7/200, Loss: 0.6931475101709371, Val_Loss: 0.6931475086212147
Epoch 8/200, Loss: 0.6931475119590763, Val_Loss: 0.6931475086212147
Epoch 9/200, Loss: 0.6931475093364722, Val_Loss: 0.6931475086212147
Epoch 10/200, Loss: 0.6931475087404257, Val_Loss: 0.6931475086212147
Epoch 11/200, Loss: 0.6931475090980539, Val_Loss: 0.6931475086212147
Epoch 12/200, Loss: 0.693147508382798, Val_Loss: 0.6931475086212147
Epoch 13/200, Loss: 0.6931475099325184, Val_Loss: 0.6931475086212147
Epoch 14/20

In [10]:
torch.save(best_model.state_dict(), "cs_model.pth")

In [11]:
model = GAT(in_channels=in_channels, hidden_channels=hidden_channels, out_channels=out_channels, heads=heads,num_layers=num_layers,dropout=dropout)
# model.load_state_dict(torch.load('model_400_500.pth'))
# model.to(device)
if(device=='cuda'):
    model.load_state_dict(torch.load('cs_model.pth'))
    print("hhh1")
else:
    model.load_state_dict(torch.load('cs_model.pth',map_location='cpu'))
    print("hhh")
    
model.to(device)

hhh


GAT(
  (convs): ModuleList(
    (0): GATConv(1, 16, heads=4)
    (1-2): 2 x GATConv(64, 16, heads=4)
    (3): GATConv(64, 8, heads=1)
  )
)

In [12]:
test_dataset = torch.load('test_dataset')
print(test_dataset[2]['y'])
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [17]:
# 计算测试集误差
model.eval()
test_acc = 0
test_recall = 0
test_f1 = 0
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        pred = model.edge_prediction(batch)
        # print(torch.sigmoid_(pred))
        # pred = torch.round(torch.sigmoid_(pred));
        pred = (torch.sigmoid(pred) >= 0.5).float()
        
        pred_np = pred.cpu().detach().numpy().flatten()
        true_np = batch.y.cpu().detach().numpy().flatten()
        
        # 计算准确率
        test_acc += accuracy_score(pred_np, true_np)/len(test_loader)
        
        # 计算召回率
        test_recall += recall_score(pred_np, true_np, zero_division=1) / len(test_loader)
        
        # 计算F1分数
        test_f1 += f1_score(pred_np, true_np, zero_division=1) / len(test_loader)

print(f'Test Acc: {test_acc}')
print(f'Test Recall: {test_recall}')
print(f'Test F1: {test_f1}')

Test Acc: 0.07970422680610328
Test Recall: 0.07970356386684863
Test F1: 0.14753443851169837
